In [1]:
import pandas as pd
from tabulate import tabulate
from custom_func import *

In [4]:
# biz; createDate, єдрпоу з казначейства
biz_createdate = pd.read_excel(f"{INPUTS_PATH}/P2/P02_007.xlsx")

# biz; createDate, єдрпоу з disposers
biz_createdate_disposers = pd.read_excel(f"{INPUTS_PATH}/P2/P02_007_disposers_timecreate.xlsx")

# biz; dateModified; єдрпоу з казначейства
biz_datemodified = pd.read_excel(f"{INPUTS_PATH}/P2/P02_007_dateModified.xlsx")
biz_datemodified["dateModified"] = pd.to_datetime(biz_datemodified["dateModified"], format="%d.%m.%Y")
biz_datemodified = biz_datemodified.loc[biz_datemodified["dateModified"].between("2020-04-01", "2020-06-30")].copy()
# bipro
pro = pd.read_excel(f"{INPUTS_PATH}/P2/P02_007_prozorro.xlsx")

# Counts

In [31]:
pro["Статуси процедур"].unique()

array(['Період уточнень', 'Завершена закупівля', 'Очікування пропозицій',
       'Активна закупівля', 'Відмінена закупівля',
       'Кваліфікація переможця', 'Закупівля не відбулась',
       'Період аукціону', 'Пропозиції розглянуто', 'Прекваліфікація',
       'Прекваліфікація (період оскаржень)'], dtype=object)

In [21]:
def count_tenders(data, label, procedure=False):
    
    _columns = {"Ідентифікатор лота": "tenderId", "Статуси процедур", "status"}
    df = data.copy()
    if procedure:
        df = df.loc[df["status"].isin(["complete", "Завершена закупівля"])]
        
    g = df.rename(columns=_columns)\
            .groupby("region")["tenderId"].size()\
            .rename(f"{label}_size")
    return g

In [22]:
g1 = count_tenders(biz_createdate, "biz_createdate")
g2 = count_tenders(biz_createdate_disposers, "biz_createdate_disposers")

g3 = count_tenders(biz_datemodified, "biz_datemodified")
g4 = count_tenders(pro, "pro")

In [23]:
# усі 
counts1 = pd.concat([g4, g1, g2, g3], axis=1)

# openbudget vs казначейство
counts2 = pd.concat([g1, g2], axis=1)

In [24]:
print(tabulate(counts1, tablefmt="pipe", headers="keys"))

| region            |   pro_size |   biz_createdate_size |   biz_createdate_disposers_size |   biz_datemodified_size |
|:------------------|-----------:|----------------------:|--------------------------------:|------------------------:|
| Івано-Франківська |       2905 |                  2804 |                            2869 |                    2941 |
| Волинська         |       3521 |                  3447 |                            3424 |                    3571 |
| Вінницька         |       4258 |                  4143 |                            4373 |                    4551 |
| Дніпропетровська  |       5404 |                  5214 |                            5406 |                    6062 |
| Донецька          |       5694 |                  5383 |                            5592 |                    5712 |
| Житомирська       |       3454 |                  3278 |                            3379 |                    3598 |
| Закарпатська      |       1956 |              

In [25]:
counts2["diff"] = counts2["biz_createdate_size"] - counts2["biz_createdate_disposers_size"]
print(tabulate(counts2, tablefmt="pipe", headers="keys"))

| region            |   biz_createdate_size |   biz_createdate_disposers_size |   diff |
|:------------------|----------------------:|--------------------------------:|-------:|
| Івано-Франківська |                  2804 |                            2869 |    -65 |
| Волинська         |                  3447 |                            3424 |     23 |
| Вінницька         |                  4143 |                            4373 |   -230 |
| Дніпропетровська  |                  5214 |                            5406 |   -192 |
| Донецька          |                  5383 |                            5592 |   -209 |
| Житомирська       |                  3278 |                            3379 |   -101 |
| Закарпатська      |                  1908 |                            1956 |    -48 |
| Запорізька        |                  3494 |                            3536 |    -42 |
| Київська          |                  2669 |                            2776 |   -107 |
| Кіровоградська    |

In [27]:
g1 = count_tenders(biz_createdate, "biz_createdate", procedure=True)
g2 = count_tenders(biz_createdate_disposers, "biz_createdate_disposers", procedure=True)
mask = pd.concat([g1, g2], axis=1)

mask["diff"] = mask["biz_createdate_size"] - mask["biz_createdate_disposers_size"]
print(tabulate(mask, tablefmt="pipe", headers="keys"))

| region            |   biz_createdate_size |   biz_createdate_disposers_size |   diff |
|:------------------|----------------------:|--------------------------------:|-------:|
| Івано-Франківська |                  2674 |                            2734 |    -60 |
| Волинська         |                  3250 |                            3262 |    -12 |
| Вінницька         |                  3703 |                            3916 |   -213 |
| Дніпропетровська  |                  3909 |                            4055 |   -146 |
| Донецька          |                  4816 |                            5014 |   -198 |
| Житомирська       |                  2821 |                            2903 |    -82 |
| Закарпатська      |                  1838 |                            1881 |    -43 |
| Запорізька        |                  2666 |                            2745 |    -79 |
| Київська          |                  2473 |                            2570 |    -97 |
| Кіровоградська    |

# Sums

In [34]:
def summarize_tenders(data, label, procedure=False):
    _columns = {"Очікувана вартість" : "tenderValue", "Статуси процедур": "status"}
    df = data.copy()
    if procedure:
        df = df.loc[df["status"].isin(["complete", "Завершена закупівля"])]
        
    g = df.rename(columns=_columns)\
            .groupby("region")["tenderValue"].sum()\
            .rename(f"{label}_sum")
    return g

In [35]:
g1 = summarize_tenders(biz_createdate, "biz_createdate")
g2 = summarize_tenders(biz_createdate_disposers, "biz_createdate_disposers")

g3 = summarize_tenders(biz_datemodified, "biz_datemodified")
g4 = summarize_tenders(pro, "pro")

In [36]:
# усі 
sums1 = pd.concat([g4, g1, g2, g3], axis=1)

# openbudget vs казначейство
sums2 = pd.concat([g1, g2], axis=1)

In [37]:
print(tabulate((sums1 / 1_000_000).round(2), tablefmt="pipe", headers="keys"))

| region            |   pro_sum |   biz_createdate_sum |   biz_createdate_disposers_sum |   biz_datemodified_sum |
|:------------------|----------:|---------------------:|-------------------------------:|-----------------------:|
| Івано-Франківська |   2116.36 |              2112.51 |                        5329.7  |                3104.51 |
| Волинська         |    948.47 |               772.9  |                         191.18 |                 994.86 |
| Вінницька         |   1065.59 |              1062.64 |                        2090.68 |                2034.31 |
| Дніпропетровська  |   3107.75 |              3092.52 |                       34466.5  |                3912.67 |
| Донецька          |   2601.91 |              2593.84 |                        4655.53 |                3584.86 |
| Житомирська       |   1989.41 |              1990.72 |                        4202.33 |                2515.12 |
| Закарпатська      |   2273.67 |              2272.97 |                        

In [38]:
print(tabulate((sums2 / 1_000_000).round(2), tablefmt="pipe", headers="keys"))

| region            |   biz_createdate_sum |   biz_createdate_disposers_sum |
|:------------------|---------------------:|-------------------------------:|
| Івано-Франківська |              2112.51 |                        5329.7  |
| Волинська         |               772.9  |                         191.18 |
| Вінницька         |              1062.64 |                        2090.68 |
| Дніпропетровська  |              3092.52 |                       34466.5  |
| Донецька          |              2593.84 |                        4655.53 |
| Житомирська       |              1990.72 |                        4202.33 |
| Закарпатська      |              2272.97 |                        4518.22 |
| Запорізька        |              1146.37 |                        1882.44 |
| Київська          |              1913.28 |                        2045.15 |
| Кіровоградська    |               819.98 |                         718.65 |
| Луганська         |               951.04 |                    

In [39]:
g1 = summarize_tenders(biz_createdate, "biz_createdate", procedure=True)
g2 = summarize_tenders(biz_createdate_disposers, "biz_createdate_disposers", procedure=True)
mask = pd.concat([g1, g2], axis=1)

mask["diff"] = mask["biz_createdate_sum"] - mask["biz_createdate_disposers_sum"]
print(tabulate(mask / 1_000_000, tablefmt="pipe", headers="keys"))

| region            |   biz_createdate_sum |   biz_createdate_disposers_sum |         diff |
|:------------------|---------------------:|-------------------------------:|-------------:|
| Івано-Франківська |             252.014  |                       287.734  |   -35.72     |
| Волинська         |             147.438  |                        96.9883 |    50.4498   |
| Вінницька         |             471.762  |                       609.34   |  -137.579    |
| Дніпропетровська  |             991.724  |                      1019.82   |   -28.0985   |
| Донецька          |             831.587  |                      2227.8    | -1396.21     |
| Житомирська       |             562.323  |                       636.928  |   -74.6046   |
| Закарпатська      |             384.611  |                       445.241  |   -60.6294   |
| Запорізька        |             482.837  |                       350.535  |   132.302    |
| Київська          |             564.083  |                       580